# User Engagement Analysis

In [3]:
import pandas as pd
import numpy as np
import os
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append("..")
from models.all_models import MLmodels as md
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D

md = md()


In [4]:
warnings.filterwarnings('ignore')


In [19]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) 
scripts_dir = os.path.join(parent_dir,"scripts") 
data_path = os.path.join(parent_dir,"data","telecom_clean_data.csv")
data_path2 = os.path.join(parent_dir,"data","user_overview_clean_data.csv")
data_path3 = os.path.join(parent_dir,"data","session_data.csv")
sys.path.insert(1, scripts_dir)

In [27]:
user_engegment_data=pd.read_csv(data_path3)

In [28]:
user_engegment_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106471 entries, 0 to 106470
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   MSISDN/Number        106471 non-null  float64
 1   Num_of_xDR_sessions  106471 non-null  int64  
 2   session_duration     106471 non-null  float64
 3   Handset Type         106471 non-null  object 
 4   AVG RTT (S)          106471 non-null  float64
 5   AVG Bearer TP        106471 non-null  float64
 6   sec with vol<6.25kb  106471 non-null  float64
 7   Total_transmit       106471 non-null  float64
 8   Other_transmit       106471 non-null  float64
 9   Gaming               106471 non-null  float64
 10  Netflix              106471 non-null  float64
 11  Youtube              106471 non-null  float64
 12  Email                106471 non-null  float64
 13  Google               106471 non-null  float64
 14  Social Media         106471 non-null  float64
dtypes: float64(13), i

### Showing top 10 users

In [32]:
engagment_metrices=['Num_of_xDR_sessions','session_duration','Total_transmit']

In [38]:
for cols in range(len(engagment_metrices)):
    print("\nTop 10 customers based on "+engagment_metrices[cols]+"\n")
    print(user_engegment_data.sort_values(by=engagment_metrices[cols], ascending=False).loc[:,['MSISDN/Number',engagment_metrices[cols]]].head(10))


Top 10 customers based on Num_of_xDR_sessions

       MSISDN/Number  Num_of_xDR_sessions
13467   3.362632e+10                   18
6404    3.361489e+10                   17
13123   3.362578e+10                   17
36909   3.365973e+10                   16
92584   3.376054e+10                   15
76077   3.367588e+10                   15
64883   3.366716e+10                   13
1270    3.360452e+10                   12
92239   3.376041e+10                   12
658     3.360313e+10                   12

Top 10 customers based on session_duration

       MSISDN/Number  session_duration
13123   3.362578e+10         18553.754
6404    3.361489e+10          9966.898
92584   3.376054e+10          9279.434
13467   3.362632e+10          8791.927
64883   3.366716e+10          8744.914
50096   3.366284e+10          6614.270
56951   3.366469e+10          6288.730
658     3.360313e+10          6287.761
65787   3.366746e+10          5649.882
92239   3.376041e+10          5321.667

Top 10 customer

In [ ]:
user_engegment_data=

# Preprocessing User Engagment Data

In [ ]:
#top_ten engagment before Normalization 

In [ ]:
#apply Noramlization 

In [ ]:
#top_ten engagment after Normalization 

In [ ]:
#k-means